In [2]:
import os
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from sklearn.linear_model import LogisticRegression

In [5]:
# Load original zhenyi files

cdr_input = '/sc/arion/projects/roussp01b/zhenyi/CDR-antagonist/results/output/2.CORRELATION_MATRICES/PsychADgrant_imputed_features_050124/eur.microglia.predict.individual_centered.csv.gz'
cdr_input = pd.read_csv(cdr_input)
cdr_input.iloc[:10, :]

,gwas,feature,zscore,model_ID
0,A-ROS-RS001020-BR-RUS-a0883,ENSG00000000457,0.183905,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
1,G-MSBB-MB000002-BR-MSBB-68416,ENSG00000000457,-0.057906,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
2,G-MSBB-MB000003-BR-MSBB-68419,ENSG00000000457,0.119793,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
3,G-MSBB-MB000004-BR-MSBB-68420,ENSG00000000457,-0.006079,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
4,G-MSBB-MB000005-BR-MSBB-68422,ENSG00000000457,-0.147502,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
5,G-MSBB-MB000006-BR-MSBB-68423,ENSG00000000457,0.010224,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
6,G-MSBB-MB000009-BR-MSBB-68427,ENSG00000000457,-0.010301,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
7,G-MSBB-MB000010-BR-MSBB-68428,ENSG00000000457,0.194538,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
8,G-MSBB-MB000011-BR-MSBB-68430,ENSG00000000457,0.050142,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...
9,G-MSBB-MB000012-BR-MSBB-68431,ENSG00000000457,0.234455,IC: Microglia :: Pseudobulk (sn) :: Genes :: P...


In [13]:
# Load the ID mappings dataframe
mypath = '/sc/arion/projects/roussp01a/deepika/merging_psychAD_SNParray_WGS/common_variants_psychAD/ancestry_pca_psychAD_1429_samples/psychAD_20PC_3_methods_ancestry.tsv'
idmap = pd.read_csv(mypath, sep='\t')
idmap.iloc[:100,:]

,IID,mod_IIDs,SubID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,...,PC18,PC19,PC20,superpop_QDA,superpop_elliposid_3SD,admixture_EUR,admixture_EAS,admixture_AMR,admixture_SAS,admixture_AFR
0,0_MSSM_109,MSSM_109,M87879,-0.014733,0.037377,0.007169,-0.000278,-0.001409,0.001798,0.001846,...,0.001228,-0.000636,-0.001462,EUR,AMR,0.946719,0.000010,0.000010,0.032090,0.021170
1,0_MSSM_112,MSSM_112,M79245,-0.016455,-0.025708,-0.016513,-0.011320,0.000256,0.000913,0.000004,...,-0.000514,-0.000602,0.000253,EUR,EUR,0.986113,0.000010,0.000010,0.013857,0.000010
2,0_MSSM_115,MSSM_115,M99132,-0.014894,0.039530,0.007221,0.001165,-0.000251,0.000213,0.002709,...,-0.001538,-0.000916,0.003289,EUR,EUR,0.950129,0.000010,0.000010,0.025199,0.024652
3,0_MSSM_12,MSSM_12,M36086,-0.014633,0.040327,0.006924,-0.000166,-0.000822,0.000825,0.002728,...,-0.001225,0.001727,0.001646,EUR,AMR,0.930907,0.000688,0.000010,0.028691,0.039705
4,0_MSSM_129,MSSM_129,M12876,-0.000705,-0.013395,0.013946,0.026244,0.005344,-0.001799,0.008364,...,0.006893,0.005157,0.005719,AMR,AMR,0.573600,0.004563,0.230169,0.011065,0.180603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,114448,114448,M23048,-0.014351,0.037364,0.007530,-0.001087,0.000230,-0.003277,0.000880,...,-0.001058,-0.001592,0.004038,EUR,EUR,0.955767,0.012837,0.000010,0.000010,0.031376
96,114535,114535,M5101,-0.016015,-0.021790,-0.016243,-0.003197,0.000231,0.000591,-0.001051,...,0.000242,-0.000476,0.001189,EUR,EUR,0.988060,0.000010,0.011910,0.000010,0.000010
97,114538,114538,M63628,-0.016405,-0.025711,-0.016759,-0.007951,0.001262,-0.001020,-0.002753,...,0.000681,-0.001559,0.000688,EUR,EUR,0.983220,0.000010,0.008769,0.007991,0.000010
98,115629,115629,M24459,-0.016462,-0.027694,-0.018774,-0.007501,0.001381,-0.003144,-0.002811,...,-0.000072,-0.002500,0.001205,EUR,EUR,0.997002,0.000010,0.002968,0.000010,0.000010
